In [1]:
import qibo
import torch
import numpy as np
from qibo import Circuit, gates, models
from qibo.optimizers import sgd
from qibo.noise import IBMQNoiseModel

from qibochem.driver.molecule import Molecule
from qibochem.ansatz import hf_circuit, ucc_circuit
from qibochem.measurement import expectation, expectation_from_samples
from qibochem.ansatz import he_circuit

In [2]:
qibo.set_backend("numpy")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bohr_angs = 0.529177210903

[Qibo 0.2.11|INFO|2024-10-15 11:52:28]: Using numpy backend on /CPU:0


In [3]:
def h2(x):
    molecule = Molecule([('H', (0, 0, 0)), ('H', (0, 0, x))])
    molecule.run_pyscf()
    return molecule

In [4]:
molecule = h2(0.7)
hamiltonian = molecule.hamiltonian()

converged SCF energy = -1.11734903499028


In [5]:
nlayers = 1
nqubits = molecule.nso # Number of molecular spin-orbitals considered for the molecule
ntheta = 2 * nqubits * nlayers
circuit = he_circuit(nqubits, nlayers)
print(circuit.draw())

q0: ─RY─RZ─o─────Z─
q1: ─RY─RZ─Z─o───|─
q2: ─RY─RZ───Z─o─|─
q3: ─RY─RZ─────Z─o─


In [6]:
parameters = {
    "t1": 10e-6,
    "t2": 5e-6,
    "gate_times" : (40*1e-9, 150*1e-9),
    "excited_population" : 0,
    "depolarizing_one_qubit" : 0,
    "depolarizing_two_qubit": 0,
    "readout_one_qubit" : {"0": 0, "1": 0},
}

noise_model = IBMQNoiseModel()
noise_model.from_dict(parameters)
noisy_circuit = noise_model.apply(circuit)
print(noisy_circuit.draw())

q0: ─RY─D─TR─RZ─D─TR─o─D─TR───────────────Z─D─TR─
q1: ─RY─D─TR─RZ─D─TR─Z─D─TR─o─D─TR────────|─|────
q2: ─RY─D─TR─RZ─D─TR────────Z─D─TR─o─D─TR─|─|────
q3: ─RY─D─TR─RZ─D─TR───────────────Z─D─TR─o─D─TR─


In [7]:
noisy_circuit.density_matrix = True
vqe = models.VQE(noisy_circuit, hamiltonian)
best, params, extra = vqe.minimize(np.random.uniform(0, 2 * np.pi, ntheta))
best, params

(-1.0719941278086664,
 array([ 3.14164909, -0.57531307,  3.14155607,  4.67844406,  6.28318531,
         2.00814184,  6.28318531,  6.2470202 ]))

In [65]:
import random
def cost(x, theta):
    # theta, x = params[1:], params[0]
    molecule = h2(x)
    hamiltonian = molecule.hamiltonian()
    circuit = he_circuit(nqubits, nlayers)
    circuit.set_parameters(theta)
    samples = expectation_from_samples(circuit, hamiltonian, n_shots=32725)
    exact = expectation(circuit, hamiltonian)
    print(exact, samples)
    return samples
    # exact = hamiltonian.expectation(circuit().state())
    # print(exact)
    # return exact

In [18]:
cost(x-0.725, theta)

TypeError: unsupported format string passed to numpy.ndarray.__format__

In [19]:
# theta = torch.randn(ntheta, dtype=torch.float32, requires_grad=True, device=device)
# x = torch.tensor([0.725], dtype=torch.float32, requires_grad=True, device=device)
# 
# learning_rate_theta = 0.001
# learning_rate_x = 0.5
# 
# optimizer_theta = torch.optim.SGD([theta], lr=learning_rate_theta)
# optimizer_x = torch.optim.SGD([x], lr=learning_rate_x)

# theta = np.random.uniform(0, 2 * np.pi, ntheta)
theta = np.array([3.14159265e+00, 3.04414775e+00, 9.42477674e+00, 7.51217906e+00,
        5.45738711e-11, 6.15682265e+00, 9.43331554e-12, 9.40210740e+00])
x = np.array([3.14159265e+00, 3.04414775e+00, 9.42477674e+00, 7.51217906e+00,
        5.45738711e-11, 6.15682265e+00, 9.43331554e-12, 9.40210740e+00])
# params = np.concatenate((x, theta))

In [67]:
from scipy.optimize import minimize, minimize_scalar
result = minimize_scalar(cost, bounds=(0.2, 3), args=theta, options={'maxiter': 100})

converged SCF energy = -0.982865947996369
-0.982865947996216 -0.9829674952127943
converged SCF energy = -0.798556290177711
-0.7985562901776074 -0.5276150346349338
converged SCF energy = -1.1003521144956
-1.1003521144954052 -1.1007000790530486
converged SCF energy = -1.1164162895382
-1.1164162895379721 -1.1156767534509455
converged SCF energy = -1.11324566742259
-1.1132456674223596 -1.1128925882868852
converged SCF energy = -1.11740487718727
-1.117404877187048 -1.1170193715067347
converged SCF energy = -1.11745534196526
-1.1174553419650373 -1.1168886125277573
converged SCF energy = -1.11391235693086
-1.1139123569306482 -1.1133609271109175
converged SCF energy = -1.11662276462111
-1.1166227646208975 -1.1174346177276964
converged SCF energy = -1.11662700569033
-1.1166270056901162 -1.1165992980351387
converged SCF energy = -1.11578465445779
-1.1157846544575802 -1.1160238700433656
converged SCF energy = -1.11633307337743
-1.116333073377215 -1.11716832567619
converged SCF energy = -1.1165166

In [55]:
dict(result)['x']

1.2695048315002941

In [35]:
def finite_diff(f, x, delta=0.01):
    """Compute the central-difference finite difference of a function"""
    gradient = []

    for i in range(len(x)):
        shift = np.zeros_like(x)
        shift[i] += 0.5 * delta
        res = (f(x + shift) - f(x - shift)) * delta**-1
        gradient.append(res)

    return gradient


def grad_x(params, x):
    grad_h = finite_diff(lambda x: h2(x).hamiltonian(), x)
    circuit.set_parameters(params)
    grad = [expectation_from_samples(circuit, obs, n_shots=1024) for obs in grad_h]
    return np.array(grad)

In [36]:
energy = []
bond_length = []

for n in range(100):
    # -------------------------
    # Optimize the circuit parameters (theta)
    # -------------------------
    optimizer_theta.zero_grad()
    energy_theta = cost(theta.detach().numpy(), x.detach().numpy()[0])
    energy_tensor = torch.tensor(energy_theta, requires_grad=True)
    energy_tensor.backward()
    optimizer_theta.step()

    # -------------------------
    # Optimize the nuclear coordinates (x)
    # -------------------------
    optimizer_x.zero_grad()
    energy_x = cost(theta.detach().numpy(), x.detach().numpy()[0])
    energy_tensor = torch.tensor(energy_x, requires_grad=True)
    energy_tensor.backward()
    optimizer_x.step()

    # Record energy and bond length
    with torch.no_grad():
        current_energy = cost(theta.detach().numpy(), x.detach().numpy()[0])
        energy.append(current_energy)
        current_bond_length = x.item()
        bond_length.append(current_bond_length)

    # Logging every 4 steps
    if n % 4 == 0:
        print(f"Step = {n},  E = {current_energy:.8f} Ha,  bond length = {current_bond_length:.5f} A")


converged SCF energy = -1.11734326912258
0.4943398242857345 0.4946501724921198
converged SCF energy = -1.11734326912258
0.4943398242857345 0.49293336210349376
converged SCF energy = -1.11734326912258
0.4943398242857345 0.4929761809615562
Step = 0,  E = 0.49433982 Ha,  bond length = 0.72500 A
converged SCF energy = -1.11734326912258
0.4943398242857345 0.496714250402336
converged SCF energy = -1.11734326912258
0.4943398242857345 0.4929635669042011
converged SCF energy = -1.11734326912258
0.4943398242857345 0.4937777227067163
converged SCF energy = -1.11734326912258
0.4943398242857345 0.4934692775256376
converged SCF energy = -1.11734326912258
0.4943398242857345 0.49612182892197493
converged SCF energy = -1.11734326912258
0.4943398242857345 0.49427959917907743
converged SCF energy = -1.11734326912258
0.4943398242857345 0.49246195226912143
converged SCF energy = -1.11734326912258
0.4943398242857345 0.49462095311381044
converged SCF energy = -1.11734326912258
0.4943398242857345 0.4906441149

In [37]:
energy

[0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242857345,
 0.4943398242